### Environment and Global Configuration

In [67]:
import os

In [68]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0

## A simple ChatBot

In [69]:
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

Instanciating a model

In [70]:
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

A System Prompt and a query

In [71]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a succinct and clear response.
"""

In [72]:
request = "I want to invest in the tech sector. What are the best options?"

response = base_model.invoke(
    [
        SystemMessage(BASE_PROMPT),
        HumanMessage(request),
    ]
)

In [74]:
Markdown(response.content)

Investing in the tech sector can be a great opportunity due to its growth potential. Here are some options to consider:

1. **Tech ETFs**: Exchange-Traded Funds like the Invesco QQQ Trust (QQQ) or the Technology Select Sector SPDR Fund (XLK) provide diversified exposure to a range of tech companies.

2. **Large Cap Tech Stocks**: Consider investing in established companies such as Apple (AAPL), Microsoft (MSFT), Amazon (AMZN), and Alphabet (GOOGL). These companies have strong fundamentals and a history of growth.

3. **Emerging Tech Stocks**: Look into smaller companies or startups in areas like artificial intelligence, cybersecurity, or cloud computing. Stocks like Palantir Technologies (PLTR) or CrowdStrike (CRWD) may offer high growth potential.

4. **Venture Capital or Private Equity**: If you have a higher risk tolerance, consider investing in venture capital funds that focus on tech startups.

5. **Sector-Specific Funds**: Mutual funds that focus on technology, such as the T. Rowe Price Global Technology Fund, can provide professional management and diversification.

6. **REITs in Tech**: Real Estate Investment Trusts that focus on data centers or tech infrastructure, like Digital Realty (DLR), can be a way to invest in the tech sector indirectly.

Before investing, assess your risk tolerance, investment horizon, and do thorough research or consult with a financial advisor to tailor your investment strategy.

## Retrieval Augmented Generation (RAG)

In [75]:
import pickle

from IPython.display import Markdown
from langchain_chroma import Chroma
from langchain_core.documents import Document

In [76]:
EMBEDDING_MODEL = "text-embedding-3-small"
RETRIEVAL_K = 3

### Vector Database and Retrieval

Some utility functions for document handling

In [77]:
def load_documents(pickle_filepath: str) -> list[Document]:
    """Load documents from a pickle file."""
    with open(pickle_filepath, "rb") as file:
        return pickle.load(file)


def initialize_vector_store(document_chunks: list[Document]) -> Chroma:
    """Reset the Chroma collection and initialize a vector store using document chunks."""
    Chroma().reset_collection()
    embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    return Chroma.from_documents(documents=document_chunks, embedding=embedding_model)

Load documents from pickle file

In [78]:
data_dir = "../data/"
data_file = "bloomberg_financial_news.pkl"

documents = load_documents(os.path.join(data_dir, data_file))

An example document

In [79]:
# TODO: change example doc for something more neutral
documents[0].metadata

{'Headline': 'Cameron Wants Access to ‘Depraved’ Child Abuse Websites Blocked',
 'Link': 'http://www.bloomberg.com/news/2013-07-21/cameron-wants-access-to-depraved-child-abuse-websites-blocked.html'}

In [81]:
Markdown(documents[0].page_content)

Internet companies need to act to block access to “depraved and disgusting” child abuse images or face legal controls, British Prime Minister David Cameron will say. “We need to have very, very strong conversations with those companies about saying ‘No, you shouldn’t provide results for some terms that are so depraved and disgusting,’” Cameron will say in an interview to be broadcast by the British Broadcasting Corp. today “I think there’s going to be a big argument there, and if we don’t get what we need we’ll have to look at legislation.” He said the extra steps he favors represents “an argument on behalf of Britain’s parents and children I’m prepared to have.” During his interview on the “Andrew Marr” program, Cameron also said he would like to see more restrictions on access to legal pornography that can be seen online by children. “There are rules about what films you can see in a cinema, what age you have to be to buy alcohol or cigarettes,” Cameron will say, according to a transcript of the interview provided by the BBC. “But on the internet, there aren’t those rules, so we need to help parents with control.”

Initialize vector store and create a retriever.

In [82]:
vector_store = initialize_vector_store(documents[:1000])
retriever = vector_store.as_retriever(search_kwargs={"k": RETRIEVAL_K})

Retrieve documents

In [83]:
retriever.invoke("tech sector market trends")

[Document(id='e1c6af54-7357-4923-bf07-265d81de5f35', metadata={'Headline': 'Japan’s IPOs to Jump 67% on Abenomics Rally, Adachi Says', 'Link': 'http://www.bloomberg.com/news/2013-03-11/japan-s-ipos-to-jump-67-on-abenomics-rally-adachi-says.html'}, page_content='Initial public offerings in Japan may jump 67 percent this year as Prime Minister Shinzo Abe’s policies boost stock prices and make it easier to sell shares, according to the Japan Venture Capital Association. New listings may rise to 80 this year from 48 in 2012, association chairman Toshihisa Adachi said. Information technology , social networking and gaming firms will make up most of the offerings, he said, without naming specific companies. Broadleaf Co., a software developer , and Water Direct Corp. (2588) , a drinking water delivery service, are among 10 companies scheduled to list in the next month amid a share rally that has boosted the Nikkei 225 Stock Average 38 percent since the end of October. IPOs have increased eve

### Augmented LLM

### Creating a tool

Create retrieval tool

In [84]:
from langchain_core.tools import tool


@tool
def retrieval(retrieval_query: str) -> list[Document]:
    """Retrieve documents based on a query."""
    return retriever.invoke(retrieval_query)

Biding to tool

In [86]:
RAG_PROMPT = """
You are a Financial Analyst with access to a Bloomberg Financial News database.

Query the database to help the client with their request. Give a succinct and clear response based on the information you find.
"""

tools = [retrieval]
tools_by_name = {tool.name: tool for tool in tools}
rag_model = base_model.bind_tools(tools)

In [87]:
request = "I want to invest in the tech sector. What are the best options?"

rag_response = rag_model.invoke(
    [
        SystemMessage(RAG_PROMPT),
        HumanMessage(request),
    ]
)

In [88]:
rag_response.tool_calls

[{'name': 'retrieval',
  'args': {'retrieval_query': 'best investment options in tech sector 2023'},
  'id': 'call_pzbGO0L9Cj1dFARdu6dYL694',
  'type': 'tool_call'}]

Query the tool

In [89]:
if rag_response.tool_calls:
    tool_call = rag_response.tool_calls[0]
    tool = tools_by_name[tool_call["name"]]
    documents = tool.invoke(tool_call["args"])

In [90]:
documents_str = "\n\n".join(
    [f"{doc.metadata['Headline']}\n\n{doc.page_content}\n" for doc in documents]
)

In [91]:
Markdown(documents_str)

Japan’s IPOs to Jump 67% on Abenomics Rally, Adachi Says

Initial public offerings in Japan may jump 67 percent this year as Prime Minister Shinzo Abe’s policies boost stock prices and make it easier to sell shares, according to the Japan Venture Capital Association. New listings may rise to 80 this year from 48 in 2012, association chairman Toshihisa Adachi said. Information technology , social networking and gaming firms will make up most of the offerings, he said, without naming specific companies. Broadleaf Co., a software developer , and Water Direct Corp. (2588) , a drinking water delivery service, are among 10 companies scheduled to list in the next month amid a share rally that has boosted the Nikkei 225 Stock Average 38 percent since the end of October. IPOs have increased every year since 2009, when they fell below 20 to the lowest in at least 15 years amid the financial crisis, according to data compiled by Bloomberg. “Japan’s strong equity market, boosted by optimism surrounding Abenomics, is definitely having a huge impact on the IPO market,” said Adachi in an interview. “In the past three to four years, companies with potential were largely undervalued by the market due to the tough environment.” Fourteen companies announced IPOs this year through yesterday in Japan including Kyoritsu Computer & Communications Co., a system solution provider, and drugmaker Medrx Co. Shares have rallied amid optimism Abe will take steps to end deflation. Eight IPOs have been announced this year in Hong Kong and 79 in the U.S. Softmax Listed Softmax Co. (3671) , a system software provider, was listed on the Tokyo Stock Exchange’s Mothers market today. It was untraded and poised to rise as buy orders outnumbered sell orders. The stock was bid at 2,600 yen as of 1:29 p.m. today, compared with its IPO price of 1,300 yen. Adachi is also chief executive officer of Itochu Technology Ventures Inc., an arm of Japan’s third-largest trading house that bought shares in Start Today Co. and online brokerage Kabu.Com Securities Co. before they went public. The venture capital firm has invested in 57 companies, according to its website. The Japan Venture Capital Association promotes start-ups through research, seminars, and collaboration with stock exchanges and government organizations, its website says. “The pipeline is continuing to increase, as if a levee broke,” Adachi said. “In two to three years, the number of IPOs will probably recover to about 100.”


Indian Stocks Retreat, End Four-Day Advance; Hero Leads Decline

Indian (SENSEX) stocks fell for the first time in five days as some investors judged last week’s rally may have outpaced the outlook for interest rate cuts by the central bank. The S&P BSE Sensex lost 0.4 percent to 19,608.76 at the 3:30 p.m. preliminary close in Mumbai. Volumes on the measure were 15 percent below the 30-day average. Hero Motocorp Ltd. (HMCL) , India biggest motorcycle maker, fell for the first time in five days. Tata Consultancy Services Ltd. (TCS) , India’s largest software services exporter, paced decline among its peers. The Sensex jumped 4 percent last week, its steepest weekly gain since November, amid speculation the Reserve Bank of India will cut interest rates at its policy review on March 19. That pushed the gauge’s valuation to 13.8 times projected 12-month profits , compared with 13.3 times at the end of February, data compiled by Bloomberg show. “Valuations have gone up again as expectations of a rate cut is getting built in,” Mehraboon Jamshed Irani, principal and head of private client group at Nirmal Bang Securities in Mumbai, told Bloomberg TV India today. The Sensex fell for five straight weeks through March 1, reaching a three-month low on Feb. 28, as profits at companies from Tata Motors (TTMT) to State Bank of India trailed estimates for the December quarter and a government report showed that the pace of the economic growth in the final three months of 2012 was the weakest in almost four years. Overseas funds bought a net $127 million worth of stocks on March 7, taking their net investment in local equities this year to $8.73 billion, a record for the period, data compiled by Bloomberg show. They bought a net $24.5 billion worth of shares last year, the most among 10 Asian markets tracked by Bloomberg.


Apple Will Announce Plans for Cash Hoard, Gamco Says: Tom Keene

Apple Inc. (AAPL) will outline what it plans to do with a growing pile of cash by next month, according to Howard Ward, chief investment officer at Gamco Investors Inc. (GBL)  Apple, which has been grappling with investor criticism over the handling of its $137.1 billion in cash and investments , will add $42 billion in earnings to that sum in 2013, Ward said. Greenlight Capital Inc.’s David Einhorn has been urging Cupertino, California-based Apple to issue high-yielding preferred shares to spread the funds among investors. Investors are also urging Apple to consider a higher dividend payout. “We’re going to get an announcement from the company as to how they intend to reallocate some of their cash,” Ward said in an interview today on Bloomberg Radio ’s “Surveillance” with Tom Keene. “They will put a floor under their stock at a higher price than it is today.” Apple climbed 1.4 percent to $437.87 at the close in New York, leaving the shares down 18 percent this year, compared with an 9.1 percent increase in the S&P 500 Index. Keith Goddard , president of Capital Advisors Inc., said Apple will probably announce a dividend or buyback by the end of the year, possibly as early as April, when the company is scheduled to release second-quarter earnings. “It really is mind-boggling to me that they’re being so stubborn about sitting on so much cash,” said Goddard, who said his firm owns 23,100 Apple shares. Apple has said it’s in active discussions over how to manage the cash, and that it’s considering buybacks, a bigger dividend among other options. Ward compared Apple’s shares to JPMorgan Chase & Co. (JPM) stock a year ago, after the bank announced a multibillion-dollar loss at the hands of one trader. “The stock was beaten up and it was at 30 dollars a share, and now it’s 50,” he said. Apple is the largest holding in Gamco’s (GBL) fund, Ward said.


Invoke the base model to avoid repeated queries

In [93]:
response = base_model.invoke(
    [
        SystemMessage(RAG_PROMPT),
        HumanMessage(request),
        rag_response,
        ToolMessage(content=documents_str, tool_call_id=tool_call["id"]),
    ]
)

In [94]:
Markdown(response.content)

For investing in the tech sector, consider the following options based on recent trends and developments:

1. **Initial Public Offerings (IPOs)**: The tech sector is seeing a surge in IPOs, particularly in Japan, where companies in information technology, social networking, and gaming are expected to dominate new listings. Notable upcoming IPOs include Broadleaf Co. and Water Direct Corp.

2. **Established Tech Giants**: Companies like Apple Inc. (AAPL) are also worth considering. Apple is expected to announce plans for its substantial cash reserves, which could include dividends or stock buybacks, potentially boosting its stock price.

3. **Emerging Markets**: Look into tech firms in emerging markets, such as Tata Consultancy Services Ltd. (TCS) in India, which is a major player in software services.

4. **Venture Capital Investments**: Consider investing in venture capital funds that focus on tech startups, as they are likely to benefit from the growing IPO market and innovation in the sector.

These options reflect a mix of established companies and emerging opportunities, providing a balanced approach to investing in the tech sector.